In [1]:
# CS 598 HW2
# Student: Zhi Ji, code is a modified version of professor Sirigano's work
import numpy as np
import scipy.signal
import os
import matplotlib.pyplot as plt
import h5py
import time
import copy
from random import randint

#load MNIST dat
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ) )
MNIST_data.close()
#x_train = x_train.reshape((60000,28,28))
#y_train = x_train.reshape((60000,28,28))
#x_test = x_train.reshape((60000,28,28))
#y_test = x_train.reshape((60000,28,28))

#plt.imshow(x_train[0,], cmap="gray")
#plt.show()

In [2]:
#Implementation of stochastic gradient descent algorithm
#number of inputs
num_inputs = 28*28
#number of hidden units
num_h = 100
#number of outputs
num_outputs = 10
d = 28
kx = 3
ky = 3
num_channel = 5

In [3]:
#initialize parameters
model = {}
#model['W'] = np.random.randn(num_outputs,d - ky+1,d-kx+1)*0.01
model['W'] = np.random.randn(num_outputs,d - ky+1,d-kx+1,num_channel)*0.01
model['b'] = np.zeros((num_outputs,1))
#model['k'] = np.random.randn(3,3) * 0.01
model['k'] = np.random.randn(3,3,num_channel) * 0.01
model_grads = copy.deepcopy(model)

def conv(img, k,d,ky,kx):
    
    new_img = np.zeros((d - ky + 1,d - kx + 1))
    for i in range(d - ky + 1):
        for j in range(d - kx + 1):
            new_img[i][j] = np.sum(img[i:i+ ky, j:j+ ky]*k)
                
                
    return new_img


In [4]:
def softmax_function(z):
    ZZ = np.exp(z)/np.sum(np.exp(z))
    return ZZ

def relu(z):
    return z * (z > 0)

def relu_back(z):
    return (z>0)

def sigmoid(z):
    return np.exp(z)/(1+np.exp(z))

#derivative of sigmoid
def sigmoid_back(z):
    s = 1/(1+np.exp(-z))
    return s * (1-s)

def forward(x,y,model):
    
    x = x.reshape((784,1))
    x = x.reshape((28,28))
    H = np.zeros((d - ky+1,d-kx+1,num_channel))
    Z = np.zeros((d - ky+1,d-kx+1,num_channel))
    #Z = np.dot(model['W1'], x) + model['b1']
    #Z = conv(x,model['k'],d,ky,kx)
    
    for p in range(num_channel):
        Z[:,:,p] = conv(x,model['k'][:,:,p],d, model['k'][:,:,p].shape[0], model['k'][:,:,p].shape[1])
        #Z[:,:,p] = scipy.signal.correlate2d(x,model['k'][:,:,p],mode ='valid')
        #print(Z[:,:,p].shape)
        H[:,:,p] = relu(Z[:,:,p])
   
    H = relu(Z)
    model['H'] = H
    model['Z'] = Z
    
    Utemp = np.dot(model['W'].reshape(num_outputs,np.prod(H.shape)),
                   H.reshape(np.prod(H.shape)))
    U = Utemp.reshape((num_outputs,1)) + model['b']
        
    p = softmax_function(U)
    return p
    
def backward(x,y,p, model, model_grads):
    x = x.reshape((784,1))
    x = x.reshape((28,28))
    
    dU = -1.0*p
    dU[y] = dU[y] + 1.0
    db = dU
    
    #dW = np.zeros((10,d - ky+1,d-kx+1))
    #for k in range(10):
    #    dW[k,:,:] = dU[k]*model['H']
    #dw = dU* model['H']
    #dW = np.dot(dU,model['H'])
    delta = np.zeros((d - ky+1,d-kx+1,num_channel))
    #for p in range(num_channel):
    dtemp = np.dot(model['W'].reshape(num_outputs,np.prod(model['H'].shape)).T,dU)
    dtemp = dtemp.reshape((d - ky+1,d-kx+1,num_channel))
    delta = dtemp

    H = model['H']
    Z = model['Z']
    #delta = np.dot(dU, model['W'].reshape(model['W'].shape[0],np.prod(H.shape))).reshape(H.shape)
    #actv_grad = (H>0).astype(float)
    dk = np.zeros((ky,kx,num_channel))
    
    for p in range(num_channel):
        dktemp = np.multiply( relu_back(Z[:,:,p]) ,  delta[:,:,p] )
        #print(dk[:,:,p].shape)
        #print(dktemp.shape[0])
        #print(x.shape)
        dk[:,:,p] = conv(x, dktemp, d, dktemp.shape[0], dktemp.shape[1])
        #dk[:,:,p] = scipy.signal.correlate2d(x,dktemp,mode ='valid')
    #print(multi.shape)
    #multi = np.multiply(delta, actv_grad)
    #dK = conv(x,multi,d,multi.shape[0],multi.shape[1])
    #dK = scipy.signal.correlate2d(x,multi)
    
    model_grads['W'] = np.dot(dU.reshape(num_outputs,1), H.reshape(np.prod(H.shape),1).T).reshape(model_grads['W'].shape)
    
   
    
    #model_grads['K'] = convolve(x,np.multiply(delta, actv_grad))
    
    
    #model_grads['W'] = dW  
    model_grads['b'] = db
    model_grads['k'] = dk #小写
   
    return model_grads

In [5]:
#timer
import time
time1 = time.time()
LR = .01
num_epochs = 10

for epochs in range(num_epochs):
    #Learning rate schedule
    #if (epochs > 5):
    #    LR = 0.001
    #if (epochs > 10):
    #    LR = 0.0001
    #if (epochs > 15):
    #    LR = 0.00001
    LR = 0.01
    total_correct = 0
    for n in range( len(x_train)):
        n_random = randint(0,len(x_train)-1 )
        y = y_train[n_random]
        x = x_train[n_random][:]
        p = forward(x, y, model)
        prediction = np.argmax(p)
        if (prediction == y):
            total_correct += 1
        model_grads = backward(x,y,p, model, model_grads)
        
        #update parameter
        model['W'] = model['W'] + LR*model_grads['W']
        model['b'] = model['b'] + LR*model_grads['b']
        model['k'] = model['k'] + LR*model_grads['k']
        #model['b2'] = model['b2'] + LR*model_grads['b2']
        
    print(total_correct/np.float(len(x_train) ) )
time2 = time.time()
print(time2-time1)

0.9230666666666667
0.9578833333333333


KeyboardInterrupt: 

In [6]:
######################################################
#test data
total_correct = 0
for n in range( len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    p = forward(x, y, model)
    prediction = np.argmax(p)
    if (prediction == y):
        total_correct += 1
        
#accuracy on test set
print(total_correct/np.float(len(x_test) ) )

0.9607


In [105]:
#testing convolution function
#def conv_single(x,k,i,j):
#    out = 0
#    for m in range(ky-1):
#        for n in range(kx - 1):
#            out += k[m,n]*x[i +m,j+n]
#    return out

#def conv(x,k,d,ky,kx):
#    output = np.zeros((d-ky+1,d-kx+1))
#    for i in range(d-ky+1):
#        for j in range(d-kx+1):
###            output[i,j] = conv_single(x,k,i,j)      
#    return output

#a = np.array([[1,1,1,0,0],
#    [0,1,1,1,0],
#    [0,0,1,1,1],
#    [0,0,1,1,0],
#    [0,1,1,0,0]])
#t = np.array([[1,0,1],
#    [0,1,0],
#    [1,0,1]])




#print(scipy.signal.correlate2d(a,t, mode = 'valid'))
#print(t.shape)
#print(conv_1(a,t))
#print(conv(a,t,5,3,3))

[[4 3 4]
 [2 4 3]
 [2 3 4]]
[[ 4.  3.  4.]
 [ 2.  4.  3.]
 [ 2.  3.  4.]]
